In [3]:
import cv2
import mediapipe as mp

def detect_hand_gesture(frame):
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands()

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            landmarks = [[landmark.x, landmark.y, landmark.z] for landmark in hand_landmarks.landmark]

            wrist = landmarks[0]
            index_tip = landmarks[8]
            middle_base = landmarks[12]

            dist_index_middle = ((index_tip[0] - middle_base[0])**2 + (index_tip[1] - middle_base[1])**2)**0.5
            dist_wrist_index = ((wrist[0] - index_tip[0])**2 + (wrist[1] - index_tip[1])**2)**0.5

            if dist_index_middle < 0.05:
                return "Fist"
            elif dist_wrist_index > 0.1: 
                return "Open Hand"

    return "No Hand"

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    gesture = detect_hand_gesture(frame)
    cv2.putText(frame, gesture, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow('Hand Gesture Detection', frame)

    if cv2.waitKey(1) & 0xFF == 27: 
        break

cap.release()
cv2.destroyAllWindows()
